## Data Ingestion

In [1]:
import os

In [2]:
os.chdir('../../')

In [3]:
%pwd

'/home/utpal108/dev/Upwork/Projects/Diabetic-Retinopathy-Prediction'

In [4]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    raw_dataset_dir: Path
    dataset_dir: Path

In [5]:
from diabeticRetinopathy.constants import *
from diabeticRetinopathy.utils import read_yaml, create_directories

In [6]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig: 
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_URL,
            raw_dataset_dir = Path(config.raw_dataset_dir),
            dataset_dir = Path(config.dataset_dir)
        )

        return data_ingestion_config

In [7]:
import opendatasets as od
import shutil
from PIL import Image

In [8]:
# Components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_dataset(self):
        if not os.path.exists(self.config.dataset_dir) or (not os.listdir(self.config.dataset_dir)):
            create_directories([self.config.raw_dataset_dir])
            # Download the dataset
            od.download(self.config.source_url,data_dir=self.config.raw_dataset_dir)


    # def _is_valid_image(self, image_path):
    #     try:
    #         with Image.open(image_path) as img:
    #             return img.size[0] > 0 and img.size[1] > 0
    #     except (OSError, IOError):
    #         return False
        

    def preprocess_dataset(self):
        if os.path.exists(self.config.raw_dataset_dir):
            try:
                create_directories([self.config.dataset_dir])
                for root, dirs, files in os.walk(self.config.raw_dataset_dir):
                    for file in files:
                        file_path = os.path.join(root, file)
                        
                        # Check if the file is an image and valid
                        # if file.lower().endswith(('.jpg', '.jpeg', '.png', '.csv')) and self._is_valid_image(file_path):
                        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.csv')):
                            # Extract the folder name
                            folder_name = os.path.basename(root)
                            
                            # Create the destination folder if it doesn't exist
                            destination_folder = os.path.join(self.config.dataset_dir, folder_name)
                            create_directories([destination_folder])
                            
                            # Copy the valid image to the destination folder
                            shutil.copy(file_path, os.path.join(destination_folder, file))

                shutil.rmtree(self.config.raw_dataset_dir)
            
            except Exception as e:
                raise e

In [9]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_dataset()
    data_ingestion.preprocess_dataset()
    
except Exception as e:
    raise e

2024-04-16 13:59:05,299 : diabeticRetinopathy.logger - INFO - YAML file: config/config.yaml loaded successfully
2024-04-16 13:59:05,301 : diabeticRetinopathy.logger - INFO - YAML file: params.yaml loaded successfully
2024-04-16 13:59:05,302 : diabeticRetinopathy.logger - INFO - created directory at: artifacts
2024-04-16 13:59:05,302 : diabeticRetinopathy.logger - INFO - created directory at: artifacts/data_ingestion
2024-04-16 13:59:05,303 : diabeticRetinopathy.logger - INFO - created directory at: artifacts/data_ingestion/raw_dataset


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading aptos2019-blindness-detection.zip to artifacts/data_ingestion/raw_dataset/aptos2019-blindness-detection


  0%|          | 40.0M/9.51G [00:30<2:03:39, 1.37MB/s]